In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy import stats
from matplotlib.pyplot import figure
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import warnings
import random


warnings.filterwarnings("ignore")
%matplotlib inline  

In [2]:
path = "../files/"
#sample_customer_id1 = '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'

In [3]:
customer_df = pd.read_csv(path+"customers.csv", usecols=['customer_id'])
customer_df

,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...
...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...


In [4]:
# Read Merged CSV
customer_product_df = pd.read_csv(path + "customer_product_purchase.csv", usecols=['customer_id', 'article_id', 'count'])
customer_product_df.head(2)

,customer_id,article_id,count
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,1
1,aa5c4c4d942c9f90f86dfe2a164c92f7a4d244a56af1c2...,789303001,1


In [ ]:
missing_df = pd.merge(customer_df, customer_product_df, on="customer_id", how="outer")
missing_df

In [6]:
article_similarity_df = pd.read_csv(path+"article-content-similarity.csv", usecols=['article_id','similar_articles'])
article_similarity_df.head(2)

,article_id,similar_articles
0,108775015,"218354001,372008001,494030013,538699001,565379..."
1,108775044,"218354021,355307042,372008010,494030010,494030..."


In [7]:
customers_similarity_df = pd.read_csv(path+"customer-similarity.csv", skiprows=1, names=['customer_id', 'similar_customers'])
customers_similarity_df.head(2)


,customer_id,similar_customers
0,0006d1fc72f81261e70e74249ab2c348e21a093ba40f2a...,d523f088074aeb6acffbdbb4cd5d34c80c3f5bdf5c5acd...
1,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,cdf9e4c4ef02a13302df5bab58ecc0f9d0ccedb6f0e38d...


In [8]:
def fn_similar_articles(article_id):
    lst =  article_similarity_df[article_similarity_df['article_id'] == article_id]['similar_articles'].tolist()
    s = ''
    if lst:
        s = lst[0]
    return s

fn_similar_articles(176209023)

'712295001,407354009,176209040,790820004,477163001,497642004,504667001,536720001'

In [9]:
def fn_similar_customers(customerid):
    lst =  customers_similarity_df[customers_similarity_df['customer_id'] == customerid]['similar_customers'].tolist()
    s = ''
    if lst:
        s = lst[0]
    return s


In [10]:
def fn_top_puchases(customerid):
    lst = customer_product_df[customer_product_df['customer_id'] == customerid].nlargest(3, 'count')['article_id'].tolist()
    return ",".join(str(x) for x in lst)


In [11]:
def predict_by_customers(customerid):
    articles_space_seperated = ''
    try:
        similar_customers =  customers_similarity_df[customers_similarity_df['customer_id'] == customerid]['similar_customers'].tolist()
        similar_customers_list = similar_customers[0].split(",")
        articles = []
        for customerid2 in similar_customers_list:
            t = fn_top_puchases(customerid2).split(",")
            articles.extend(t)
        articles_space_seperated = " ".join(str(x) for x in articles[0:12])
    except:
        print("Errored - ", customerid)
    return articles_space_seperated


In [12]:
def predict_by_purchase_history(customerid):
    articles_space_seperated = ''
    try:
        purchased_articles_list = fn_top_puchases(customerid).split(",")
        articles = []
        for item in purchased_articles_list:
            similar_articles =  fn_similar_articles(176209023)
            articles.extend(similar_articles.split(","))
        articles_space_seperated = " ".join(str(x) for x in articles[0:12])
    except:
        print("Errored - ", customerid)
    return articles_space_seperated
                  

In [13]:
k = 1500
arr = np.array_split(customer_df, k)

In [ ]:
for i in range(k):
    chunk = pd.DataFrame(arr[i])
    chunk['predictions_by_customer'] = chunk['customer_id'].apply(lambda x: predict_by_customers(x))    
    chunk['predictions_by_purchase_history'] = chunk['customer_id'].apply(lambda x: predict_by_purchase_history(x))
    chunk.to_csv(path+"subfiles/intermediate" + str(i) + ".csv")

Errored -  0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa
Errored -  00058ecf091cea1bba9d800cabac6ed1ae284202cdab68bec5c8429eb3271c0c
Errored -  000df4d2084d142416b8165bdd249bab8fea2393447aed8df063eb3a68b17b14


In [ ]:
#customer_df['predictions_by_customer'] = customer_df['customer_id'].apply(lambda x: predict_by_customers(x))
#customer_df

In [ ]:
#customer_df['predictions_by_purchase_history'] = customer_df['customer_id'].apply(lambda x: predict_by_purchase_history(x))
#customer_df


In [ ]:
customer_df.to_csv(path+"intermediate1.csv")